In [266]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [268]:
# !pip install contractions

In [269]:
# # In case of spellcheck
# import spacy
# import contextualSpellCheck #using Bert model
# # The idea of using BERT was to use the context when correcting non-word error (NWE) .

# nlp = spacy.load('en_core_web_sm')
# contextualSpellCheck.add_to_pipe(nlp)
# doc = nlp('Income was $9.4 milion compared to the prior year of $2.7 milion.')

# print(doc._.performed_spellCheck) #Should be True
# print(doc._.outcome_spellCheck) #Income was $9.4 million compared to the prior year of $2.7 million.

In [270]:
from collections import Counter
import os
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords_spacy
from bs4 import BeautifulSoup
import nltk
# nltk.download('omw-1.4')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import contractions
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import math
import torch.nn.functional as F

In [271]:
corpus_movie_conv = '/content/movie_conversations.txt'
corpus_movie_lines = '/content/movie_lines.txt'
max_len = 25

In [272]:
os.path.isfile(corpus_movie_lines)

True

In [273]:
with open(corpus_movie_conv,"r", encoding='iso-8859-1') as f:
  conv = f.readlines()
with open(corpus_movie_lines,"r",encoding='iso-8859-1') as f:
  lines=f.readlines()

In [274]:
conv[:3]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']\n",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']\n",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']\n"]

In [275]:
lines[:3]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n']

In [276]:
lines[0]

'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'

In [277]:
lines[0].split(" +++$+++ ")

['L1045', 'u0', 'm0', 'BIANCA', 'They do not!\n']

In [278]:
lines[0].split(" +++$+++ ")[-1]

'They do not!\n'

In [279]:
lines_dict={}
for line in lines:
    temp = line.split(" +++$+++ ")
    lines_dict[temp[0]]=temp[-1]

In [280]:
# # uncomment if you want to look at lines_dict
# lines_dict

In [281]:
#we need to expand contractions
lines_dict["L194"]

'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'

In [282]:
# import contractions
# contractions.add('mychange', 'my change')
contractions.add("'bout","about")

In [283]:
contractions.fix("'bout")

'about'

In [284]:
# Define functions for preprocessing of text data
def get_contraction(x):
    return contractions.fix(x)
def remove_punctuation(x):
    return x.translate(str.maketrans('', '', string.punctuation))

In [285]:
# Example
print("Original text: ",lines_dict["L195"])
expand_sentence = get_contraction(lines_dict["L195"]).lower()
print("expanded text: ",expand_sentence)

Original text:  Well, I thought we'd start with pronunciation, if that's okay with you.

expanded text:  well, i thought we would start with pronunciation, if that is okay with you.



In [286]:
# Example
print("sentence with punc: ",expand_sentence)
print("sentence without punc: ",remove_punctuation(expand_sentence))

sentence with punc:  well, i thought we would start with pronunciation, if that is okay with you.

sentence without punc:  well i thought we would start with pronunciation if that is okay with you



In [287]:
# expand contractions and remove the punctuations, also remove extra space in the sentence
for key, value in lines_dict.items():
    temp = remove_punctuation(get_contraction(value)).lower()
    temp = temp.split()
    temp = " ".join(temp)
    lines_dict[key]= temp

In [288]:
# controlling the expanded contractions and punctuations
lines_dict["L194"]

'can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again'

In [289]:
# We will eval the conversation texts
eval(conv[0].split(" +++$+++ ")[-1])

['L194', 'L195', 'L196', 'L197']

In [290]:
pairs = []
for con in conv:
    ids=eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        question_and_pairs = []
        if i == len(ids) - 1:
            break
        
        first = lines_dict[ids[i]].strip()
        second = lines_dict[ids[i+1]].strip()
        question_and_pairs.append(first.split()[:max_len])
        question_and_pairs.append(second.split()[:max_len])
        
        pairs.append(question_and_pairs)

In [291]:
print(f"We have {len(pairs)} pairs, which we will train on")

We have 221616 pairs, which we will train on


#### Now we create word to index dictionary
- we will gather all the unique words in dataset and get counts of them.
- we will remove words that occur less than five times, because we dont want to increase the number of vocabulary words

In [292]:
# look at question and answer conversation samples
for v, pair in enumerate(pairs):
    print(pair)
    if v==3:
        break   

[['can', 'we', 'make', 'this', 'quick', 'roxanne', 'korrine', 'and', 'andrew', 'barrett', 'are', 'having', 'an', 'incredibly', 'horrendous', 'public', 'break', 'up', 'on', 'the', 'quad', 'again'], ['well', 'i', 'thought', 'we', 'would', 'start', 'with', 'pronunciation', 'if', 'that', 'is', 'okay', 'with', 'you']]
[['well', 'i', 'thought', 'we', 'would', 'start', 'with', 'pronunciation', 'if', 'that', 'is', 'okay', 'with', 'you'], ['not', 'the', 'hacking', 'and', 'gagging', 'and', 'spitting', 'part', 'please']]
[['not', 'the', 'hacking', 'and', 'gagging', 'and', 'spitting', 'part', 'please'], ['okay', 'then', 'how', 'about', 'we', 'try', 'out', 'some', 'french', 'cuisine', 'saturday', 'night']]
[['you', 'are', 'asking', 'me', 'out', 'that', 'is', 'so', 'cute', 'what', 'is', 'your', 'name', 'again'], ['forget', 'it']]


In [293]:
word_counts = Counter()
for pair in pairs:
#     we update word counts sequentially for each pair (conversation pair)
    word_counts.update(pair[0]) #question
    word_counts.update(pair[1]) #answer

In [294]:
# word_counts

In [295]:
min_word_counts = 5
words = [ word for word in word_counts.keys() if word_counts[word] > min_word_counts]
words_map = { k:v+1 for v, k in enumerate(words)}# v=index, k=word , v+1 because start_index=1
words_map["<unknown>"]= len(words_map) +1 # unknown words is assigned to word which occurs less than five times
words_map["<start>"]= len(words_map) +1 
words_map["<end>"]= len(words_map) +1 
words_map["<pad>"]= 0

In [296]:
print(f"Total words are: {len(words_map)}")

Total words are: 18039


In [297]:
words_map["<end>"]

18038

In [298]:
with open("Wordmap_corpus.json","w") as f:
    json.dump(words_map,f)

In [299]:
def encode_question(words, words_map):
    encoded_question = [words_map.get(word, words_map["unknown"]) for word in words] + [words_map["<pad>"]] * (max_len-len(words))
    return encoded_question
#     if word does not exits, then it will return words_map["unknown"] value

In [300]:
def encode_answer(words, words_map):
    encoded_answer = [words_map["<start>"]] + [words_map.get(word, words_map["unknown"]) for word in words] + [words_map["<end>"]] + [words_map["<pad>"]] * (max_len-len(words))
    return encoded_answer

In [301]:
pairs_encoded = []
for pair in pairs:
    question = encode_question(pair[0], words_map)
    answer = encode_answer(pair[1], words_map)
    pairs_encoded.append([question, answer])

In [302]:
# Visualize the first element of this matrix
# <start> 18002
# <end> 18003
print(pairs_encoded[0])

[[1, 2, 3, 4, 5, 1624, 1624, 6, 7, 8, 9, 10, 11, 12, 1624, 13, 14, 15, 16, 17, 1624, 18, 0, 0, 0], [18037, 19, 20, 21, 2, 22, 23, 24, 1624, 25, 26, 27, 28, 24, 29, 18038, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [303]:
with open("pairs_encoded.json","w") as f:
    json.dump(pairs_encoded, f)

In [304]:
class Mydataset(Dataset):
    def __init__(self):
        path = "/content/pairs_encoded.json"
        self.pairs = json.load(open(path))
        self.dataset_size = len(self.pairs)
    
    def __getitem__(self, idx):
#         we need to convert this list containin numbers to a long tensor
        question = torch.LongTensor(self.pairs[idx][0]) 
        answer = torch.LongTensor(self.pairs[idx][1]) # this will retrieve one element from your dataset
        return question, answer
    
    def __len__(self):
        return self.dataset_size

In [305]:
train_loader = torch.utils.data.DataLoader(Mydataset(),
                                           batch_size = 100,
                                          shuffle = True,
                                          pin_memory = True)

In [306]:
question, answer = next(iter(train_loader))

In [307]:
question.shape, answer.shape
# answer has length of 27, because it has <start> and<end> token

(torch.Size([100, 25]), torch.Size([100, 27]))

In [308]:
# - sentence      : <start> I went to home last night <end>
# - answer        : <start> I went to home last night , "<start>" is expected to predict "I", "I" is expected to predict "went", "went" is expected to predict "to"
# - answer target :     I went to home last night <end>

In [309]:
def create_masks(question, answer, answer_target):
    
    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size,size)).transpose(0,1).type(dtype=torch.uint8) 
        # since we are working with 4D tensor, we need to unsequeeze it
        return mask.unsqueeze(0)
    
    question_mask = ((question !=0).int()).to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1) # (batch_size, 1, 1, max_words)
    
    answer_mask = ((answer !=0).int()).to(device)
    answer_mask = answer_mask.unsqueeze(1) # (batch_size, 1, max_words)
    answer_mask = answer_mask & subsequent_mask(answer.size(-1)).type_as(answer_mask.data) # answer.size(-1) = max_words
#     (batch_size, max_words, max_words)
    answer_mask = answer_mask.unsqueeze(1)
    answer_target_mask = (answer_target !=0).int()
    
    return question_mask, answer_mask, answer_target_mask

In [310]:
(question[0]!=0).int()

tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0], dtype=torch.int32)

#### Embeddings

In [311]:
# Create Embeddings
class Embeddings(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_len=50):
        super(Embeddings,self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.max_len = max_len
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embedding(self.vocab_size, self.embed_dim)
        self.positional_encod = self.create_positional_encoding(self.max_len, self.embed_dim) # call function after creating
        
    def create_positional_encoding(self, max_len, embed_dim):
        positional_encoding = torch.zeros(max_len, embed_dim).to(device)
        for pos in range(max_len):
            for i in range(0, embed_dim, 2):
                positional_encoding[pos, i] = math.sin(pos / math.pow(10000,2*i/embed_dim) )
                positional_encoding[pos, i+1] = math.cos(pos / math.pow(10000,2*i/embed_dim) )
#                 we need to include batch size for this
        positional_encoding = positional_encoding.unsqueeze(0) #results in shape (batch_size, max_len, embed_dim) --> (1, max_len, embed_dim),
        return positional_encoding
    
    def forward(self, encoded_words):
        embeddings = self.embed(encoded_words)*math.sqrt(self.embed_dim) # (batch_size, max_words, embed_dim)
        # embeddings retrieve its embedding vector. These vectors will then be learnt as a parameters by the model,
#         we wanna give more importance to the embeddings so we multiply is sqrt(self.embed_dim)
#         max_words = embedding.size(1)
        embeddings += self.positional_encod[:, :embeddings.size(1),] # pe will be automatically expanded to same batch_size as embeddings
        embeddings = self.dropout(embeddings)
        return embeddings

In [340]:
class Multi_Head_Attention(nn.Module):
    
    def __init__(self, heads, model_dim): # model_dim or embed_dim
        super(Multi_Head_Attention, self).__init__()
        assert model_dim % heads == 0
        self.heads = heads
        self.model_dim = model_dim
        self.d_k = self.model_dim // self.heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(self.model_dim, self.model_dim)
        self.key = nn.Linear(self.model_dim, self.model_dim)
        self.value = nn.Linear(self.model_dim, self.model_dim)
        self.concat = nn.Linear(self.model_dim, self.model_dim)
    
    def forward(self,query, key, value, mask):
        query = self.query(query) # (batch_size, max_len, model_dim) ----> (batch_size, max_len, 512)
        key = self.key(key) # (batch_size, max_len, model_dim)
        value = self.value(value) # (batch_size, max_len, model_dim)
        
#         (batch_size, max_words, 512) ---> (batch_size, max_words, 8, 64) ---> (batch_size, 8, max_words, d_k)
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3) # result shape:(batch_size, 8, max_words, h, d_k) 
        key = key.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3) # result shape:(batch_size, 8, max_words, d_k)
        value = value.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3) # result shape:(batch_size, 8, max_words, d_k)
        
#         (batch_size, 8, max_words, d_k) dot (batch_size, 8, d_k, max_words) == result shape:(batch_size, 8 , max_words, max_words)
        scores = torch.matmul(query, key.permute(0,1,3,2)) / math.sqrt(self.d_k)  # result shape:(batch_size, 8 , max_words, max_words)
        scores = scores.masked_fill(mask ==0, -1e9)
        weights = F.softmax(scores, dim = -1)
        weights = self.dropout(weights) # result shape:(batch_size, 8, max_words, max_words)
        
#         (batch_size, 8, max_words, max_words) dot value (batch_size, 8, max_words, d_k) ------> result shape: (batch_size, 8, max_words, d_k)
        context = torch.matmul(weights,value) # result shape: (batch_size, 8, max_words, d_k)
#         transpose it from (batch_size, 8, max_words, d_k) to (batch_size, max_words, 8 , d_k) and concatenate them (batch_size, max_words, 8*d_k)
        context = context.permute(0,2,1,3).contiguous() # result shape: (batch_size, max_words, 8 , d_k)
        context = context.view(context.shape[0], -1, self.heads*self.d_k) # result shape: (batch_size, max_words or max_len, 512 or model_dim = 512)
        context_interacted = self.concat(context)
        
        return context_interacted

In [341]:
class Feed_Forward(nn.Module):
    
    def __init__(self,embed_dim, middle_dim = 2048):
        super(Feed_Forward,self).__init__()
        self.embed_dim = embed_dim
        self.middle_dim = middle_dim
        self.fc_linear1 = nn.Linear(self.embed_dim, self.middle_dim) #(512, 2048) --> result shape: 2048
        self.fc_linear2 = nn.Linear(self.middle_dim, self.embed_dim) #(2048,512) --> result shape 512
        self.dropout = nn.Dropout(0.1) # dropout rate used in the original paper
    
    def forward(self, x):
        x = self.fc_linear1(x)
        x = F.relu(x) # activation function used in the original paper
        x = self.dropout(x)
        x = self.fc_linear2(x)
        return x        

#### d_model == embed_dim
- In Encoder, we only have self_attention
- In Decoder, we both have self_attention and source attention

In [342]:
class Encoder_layer(nn.Module):
    
    def __init__(self,embed_dim, heads):
        super(Encoder_layer,self).__init__()
        
        self.self_multihead_attention = Multi_Head_Attention(heads, embed_dim)
        self.feed_forward = Feed_Forward(embed_dim, middle_dim= 2048)
        self.layernorm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, embeddings, mask):
        interacted_context = self.self_multihead_attention(embeddings, embeddings, embeddings, mask)
#         in transformer, self_multihead_attention( query, key, value, mask) takes these values normally
        interacted_dropout = self.dropout(interacted_context)
        interacted_normalized = self.layernorm(interacted_dropout + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted_normalized))
        encoded_output = self.layernorm(feed_forward_out + interacted_normalized )
        return encoded_output

In [343]:
class Decoder_layer(nn.Module):
    
    def __init__(self, embed_dim, heads):
        super(Decoder_layer,self).__init__()
        self.self_multihead_attention = Multi_Head_Attention(heads, embed_dim)
        self.source_multihead_attention = Multi_Head_Attention(heads, embed_dim)
        self.feed_forward = Feed_Forward(embed_dim, middle_dim= 2048)
        self.layernorm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, embeddings, encoded_output, source_mask, target_mask):
        query = self.self_multihead_attention(embeddings, embeddings, embeddings, target_mask) # create query for source_attention
        query = self.layernorm(query + embeddings) # outputs of the self attention plus the input, which is the embeddings
#         keys and values as an input to source_attention comes from the encoded output
        source_attention = self.source_multihead_attention(query, encoded_output, encoded_output, source_mask)
        interacted_source_attention = self.dropout(source_attention)
        interacted_source_attention = self.layernorm(interacted_source_attention + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted_source_attention))
        decoded_output = self.layernorm(feed_forward_out + interacted_source_attention)
        return decoded_output

#### Build a full transformer model in py

In [344]:
class Transformer(nn.Module):
    
    def __init__(self, embed_dim, heads, num_layers, words_map):
        super(Transformer, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = len(words_map)
        self.embed  = Embeddings(self.vocab_size, self.embed_dim)
#         N=num_layers / Encoder_layers are defined in original paper, N=6
#         we have 6 Encoder_layers whose architerctures are the same but have different weights
        self.encoder = nn.ModuleList([Encoder_layer(self.embed_dim, heads) for _ in range(num_layers)]) # assembles models in list
        self.decoder = nn.ModuleList([Decoder_layer(self.embed_dim, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(self.embed_dim, self.vocab_size)
        
    def encode(self, source_words, source_mask): # source_words are questions
        source_embeddings = self.embed(source_words)
        for layer in self.encoder:
            source_embeddings = layer(source_embeddings,source_mask)
#             source_embeddings are fed into first encoder layer
#             Here, there are 6 encoder layers, and output of the first encoder layer is fed into input of the second encoder layer and so it goes.
        return source_embeddings

    def decode(self, target_words, target_mask, source_embeddings, source_mask): # source_words are questions
        target_embeddings = self.embed(target_words)
        for layer in self.decoder:
            target_embeddings = layer(target_embeddings, source_embeddings, source_mask, target_mask)
#             source_embeddings are fed into first encoder layer
#             Here, there are 6 encoder layers, and output of the first encoder layer is fed into input of the second encoder layer and so it goes.
#             Here, both encoder and decoder share the same vocabulary and same words
        return target_embeddings
    
    def forward(self, source_words, source_mask, target_words, target_mask):
        encoded = self.encode(source_words, source_mask)
        decoded = self.decode(target_words, target_mask, encoded, source_mask)
#         why do we need log_softmax?
#         we can ignore it if we train with crossentropy, we dont need to define log_softmax at all
#         But here we use KL divergence loss, therefore we need to define softmax function
        out = F.log_softmax(self.logit(decoded))
#         -------------- additional information ---------------------
#         pytorch automatically takes log and softmax of output logit if you use crossentropy
#         out = self.logit(decoded)
#         -----------------------------------------------------------
        return out

#### Now, we will define Adam optimization with warmup

In [345]:
class Adam_warmup:
    
    def __init__(self, model_size, warmup_steps, optimizer):
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0
        
    def get_lr(self):
        return self.model_size**(-0.5)*min(self.current_step**(-0.5), self.current_step*self.warmup_steps**(-1.5))
    
    def step(self):
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group["lr"] = lr   #update lr
        self.lr = lr
        self.optimizer.step() # update weights   

- w < -- w - lr*gradient
- optimizer.step()

In [369]:
class LosswithLS(nn.Module):
    def __init__(self, size, smooth):
        super(LosswithLS, self).__init__()
        self.criterion = nn.KLDivLoss(size_average= False, reduce= False) # The shape of matrix will be changed if reduce is True
        self.smooth = smooth
        self.confidence = 1 - self.smooth
        self.size = size
    
    def forward(self, prediction, target, mask):
        """
        prediction: (batch_size, max_words, vocab_size)
        target and mask: (batch_size, max_words)
        """
        prediction = prediction.view(-1,prediction.size(-1))
        target = target.contiguous().view(-1)
        mask = mask.float()
        mask = mask.view(-1)
        labels = prediction.data.clone()
        labels.fill_(self.smooth/(self.size-1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = self.criterion(prediction, labels)  # shape (batch_size*max_words, vocab_size)
        loss = (loss.sum(1)*mask).sum()/ mask.sum() # for example 1 0 1 1 0, ones are the words, 3/5
        return loss

#### Visualize the operation

-------------------------------------------------------------------------------------------------------------------------------------

In [370]:
batch_size=2
max_words=4
vocab_size=3
prediction = torch.randn(batch_size, max_words, vocab_size)
prediction.shape

torch.Size([2, 4, 3])

In [371]:
prediction = prediction.view(-1, prediction.shape[-1])
prediction.shape

torch.Size([8, 3])

In [372]:
target = torch.LongTensor(batch_size*max_words).random_(0,prediction.shape[-1])
target

tensor([0, 0, 1, 0, 1, 2, 0, 1])

In [373]:
mask= target!=0
mask.shape

torch.Size([8])

In [374]:
labels = prediction.data.clone()
labels.shape

torch.Size([8, 3])

In [375]:
labels

tensor([[-0.7088,  1.1347,  0.5537],
        [-0.1594, -1.1809, -1.6327],
        [ 0.0231, -0.7799,  0.8135],
        [-0.4532, -0.5335,  0.6476],
        [ 0.1810, -1.5008, -1.4220],
        [ 0.5921, -1.8903, -1.5533],
        [ 0.5950, -0.6280,  1.1034],
        [-1.0888, -0.7710, -0.2193]])

In [376]:
labels.fill_(0.3/3-1)

tensor([[-0.9000, -0.9000, -0.9000],
        [-0.9000, -0.9000, -0.9000],
        [-0.9000, -0.9000, -0.9000],
        [-0.9000, -0.9000, -0.9000],
        [-0.9000, -0.9000, -0.9000],
        [-0.9000, -0.9000, -0.9000],
        [-0.9000, -0.9000, -0.9000],
        [-0.9000, -0.9000, -0.9000]])

In [377]:
target

tensor([0, 0, 1, 0, 1, 2, 0, 1])

In [378]:
labels.scatter(1, target.data.unsqueeze(1), 1-0.3) # in order to use scatter, it has to be same dimension as target

tensor([[ 0.7000, -0.9000, -0.9000],
        [ 0.7000, -0.9000, -0.9000],
        [-0.9000,  0.7000, -0.9000],
        [ 0.7000, -0.9000, -0.9000],
        [-0.9000,  0.7000, -0.9000],
        [-0.9000, -0.9000,  0.7000],
        [ 0.7000, -0.9000, -0.9000],
        [-0.9000,  0.7000, -0.9000]])

In [379]:
loss = torch.randn(batch_size*max_words, vocab_size)
loss.shape

torch.Size([8, 3])

In [380]:
loss

tensor([[-0.1688,  0.7135, -0.7854],
        [ 0.8839,  0.6079, -0.7647],
        [ 0.7714, -0.2188,  1.7395],
        [-0.6832,  1.0505, -0.0190],
        [-0.3805, -0.7658, -1.0043],
        [ 0.9719, -1.1071, -0.0734],
        [ 1.9451, -0.0305, -0.1205],
        [-0.8273,  0.5719, -0.3456]])

In [381]:
loss.sum(1)  # we sum of the elements along the direction dim =1

tensor([-0.2407,  0.7270,  2.2921,  0.3483, -2.1505, -0.2086,  1.7940, -0.6010])

In [382]:
mask

tensor([False, False,  True, False,  True,  True, False,  True])

In [383]:
loss.shape, mask.shape

(torch.Size([8, 3]), torch.Size([8]))

In [384]:
loss.sum(1)*mask.float()

tensor([-0.0000,  0.0000,  2.2921,  0.0000, -2.1505, -0.2086,  0.0000, -0.6010])

In [385]:
(loss.sum(1)*mask.float()).sum() / mask.sum()

tensor(-0.1670)

-------------------------------------------------------------------------------------------------------------------------------------

In [386]:
embed_dim = 512
heads = 8
num_layers = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 1

with open("/content/Wordmap_corpus.json","r") as f:
    words_map = json.load(f)
    
transformer = Transformer( embed_dim= embed_dim,heads= heads, num_layers= num_layers, words_map= words_map)
transformer.to(device)

adam_optimizer = torch.optim.Adam(transformer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
transformer_optimizer = Adam_warmup(embed_dim, warmup_steps=4000, optimizer=adam_optimizer)
criterion = LosswithLS(size=len(words_map), smooth=0.3)

### Training function

In [387]:
def train(train_loader, transformer, criterion, epoch):
    
    transformer.train() # we need to set it training mode, because we are using dropout
    # dropout acts differently in training and testing time.
    sum_loss = 0
    count = 0
    
    for i, (question, answer) in enumerate(train_loader):
        samples = question.shape[0] # retrieve the batch size
        question = question.to(device)
        answer = answer.to(device)
        
        # at a training time: for instance sentence: <start> A man is playing with a dog <end>  -----> Sentence
        answer_input = answer[:,:-1] # last word is <end> token, we dont take it, example: <start> A man is playing with a dog -----> Input
        answer_target = answer[:, 1:] # Beginning word is <start> token, we dont take it, example: A man is playing with a dog <end>  -----> Target
        question_mask, answer_mask, answer_target_mask = create_masks(question, answer_input, answer_target)
        
        out = transformer(question, question_mask,answer_input, answer_mask) # now, we have our predictions
        loss = criterion(out, answer_target, answer_target_mask)
        #backpropagation
        transformer_optimizer.optimizer.zero_grad() # we dont want to accumulate gradients
        loss.backward()
        transformer_optimizer.step()
        sum_loss += loss.item() * samples # using item(), because we are getting value from tensor
        count += samples # at the last batch for an epoch, the count = len(train_loader)*100
        
        if i % 100 ==0:
            print("Epoch [{}][{}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), sum_loss / count))
        

### Evaluation function

In [388]:
def evaluate(transformer, question, question_mask, max_len, words_map):
    """
    This function is gonna perform greedy search decoding with a batch size of 1
    """
    rev_words_map = { index : word for word, index in words_map.items() }
    transformer.eval()
    start_token = words_map["<start>"]
#     def encode(self, source_words, source_mask):
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device) # shape: (1, 1)
    
    for step in range(max_len-1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size,size)).transpose(0,1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0)
#         def decode(self, target_words, target_mask, source_embeddings, source_mask):
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
#         decoded shape : (batch_size, max_len, vocab_size) ---> (1, 1, vocab_size)
        predictions = transformer.logit(decoded[:, -1]) # we only take the last element, outout of the work that you are about to generate
#         predictions shape: (1, vocab_size)
        _, next_word = torch.max(predictions, dim =1) # next_word shape:(1, 1)
        next_word = next_word.item()
        if next_word == words_map["<end>"]:
            break
        words = torch.cat( [words, torch.LongTensor([[next_word]]).to(device)] , dim =1) #shape: (1, step+2)
    
    #words shape will be like (1, 5), so we will get rid of first dimension
    words = words.squeeze(0) #resulting dimension is 1 dimensional tensor with 5 elements, each of which will be a predicted word.
    words = words.tolist() #convert tensor to list
    
    # build sentence
    sentence_idx = [w for w in words if w not in {words_map["<start>"]}]
    sentence = " ".join([ rev_words_map[sentence_idx[i]] for i in range(len(sentence_idx)) ])
    return sentence

In [389]:
epochs

1

In [ ]:
for epoch in range(epochs):
  train(train_loader, transformer, criterion, epoch)
  state = {"epoch": epoch, "transformer" : transformer, "transformer_optimizer": transformer_optimizer}
  torch.save(state, "checpoint_" + str(epoch) + ".tar")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [0][0/2217]	Loss: 1.184


In [1]:
# # after you train the transformer, you can choose checkpoint
checkpoint = torch.load("checkpoint_1.tar")
transformer = checkpoint["transformer"]


In [ ]:
while(1):
  question = input("Question: ")
  if question == "quit":
    break
  
  max_len = input("Enter max words to be generated: ")
  encoded_qeustion = [ words_map.get(word, words_map["<unknown>"]) for word in question.split()]
  question = torch.LongTensor(encoded_qeustion).to(device).unsqueeze(0)
  question = (question !=0).to(device).unsqueeze(1).unsqueeze(1)
  sentence = evaluate(transformer, question, question_mask, int(max_len),words_map)
  print(sentence)
